In [1]:
import pandas as pd
import altair as alt
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

# --- Configuration des Chemins ---
FINAL_DATASET_PATH = "hdfs://namenode:9000/user/mathis/datalake/nyc_final_air_weather_dataset.parquet"

# Recharger la session Spark si elle a été arrêtée
spark = SparkSession.builder.appName("Dashboard_Data_Load").getOrCreate()

# 1. Lecture du jeu de données final depuis HDFS
df_spark = spark.read.parquet(FINAL_DATASET_PATH)

print(f"Jeu de données final chargé. Nombre d'enregistrements : {df_spark.count()}")
df_spark.printSchema()

# 2. Convertir les dates en format lisible (YYYY-MM-DD)
df_spark = df_spark.withColumn(
    "DATE", 
    F.to_date(F.col("DATE_OBSERVATION").cast("string"), "yyyyMMdd")
).drop("DATE_OBSERVATION")

# 3. Collecte vers Pandas (pour Altair/Visualisation)
# ATTENTION: Cette action 'collect()' nécessite assez de RAM.
df_pandas = df_spark.toPandas()

# Nettoyage final des noms de colonnes pour Altair
df_pandas.columns = [col.replace('_MOYENNE', '').lower() for col in df_pandas.columns]

# Enregistrement du DataFrame Pandas pour analyse future si nécessaire
df_pandas.to_csv("nyc_final_dashboard_data.csv", index=False)

print("\nDonnées chargées en DataFrame Pandas (df_pandas) pour la visualisation.")

Jeu de données final chargé. Nombre d'enregistrements : 0
root
 |-- GEOJOIN_ID: string (nullable = true)
 |-- DATE_OBSERVATION: string (nullable = true)
 |-- TEMP_MOYENNE_F: double (nullable = true)
 |-- PRECIPITATION_POUCES: double (nullable = true)
 |-- VITESSE_VENT_NOEUDS: double (nullable = true)
 |-- PHENOMENES: string (nullable = true)
 |-- QA_Annual_vehicle_miles_traveled_MOYENNE: double (nullable = true)
 |-- QA_Annual_vehicle_miles_traveled_cars_MOYENNE: double (nullable = true)
 |-- QA_Annual_vehicle_miles_traveled_trucks_MOYENNE: double (nullable = true)
 |-- QA_Asthma_emergency_department_visits_due_to_PM2_5_MOYENNE: double (nullable = true)
 |-- QA_Asthma_emergency_departments_visits_due_to_Ozone_MOYENNE: double (nullable = true)
 |-- QA_Asthma_hospitalizations_due_to_Ozone_MOYENNE: double (nullable = true)
 |-- QA_Boiler_Emissions-_Total_NOx_Emissions_MOYENNE: double (nullable = true)
 |-- QA_Boiler_Emissions-_Total_PM2_5_Emissions_MOYENNE: double (nullable = true)
 |-- Q